In [17]:
from queue import PriorityQueue
from numpy.random import choice
from abc import ABC, abstractmethod
from copy import copy

In [16]:
class GameState(ABC):
    @abstractmethod
    def __init__(self):
        pass
        
    def process_triggers(self, verbose=False, abl_queue=PriorityQueue(), n_iter=0):
        """
        Function for resolving ability triggers of Team objects. 
        Ability users must be alive or have a Self Faint trigger or have a StartOfBattle. 
        Clean dead pets and shift teams to front. All changes are in-place.
        """                
        # TODO: add atk priority, plus others
        # Loop through ordered pets while queueing abilities with triggers. Note this does NOT store the team.
            
        is_shop = not isinstance(self, Battle)
        team1 = self.team1
        team2 = Team([]) if is_shop else self.team2
        
        if verbose:
            print(f'Processing: \n{self!s}')
                    
        # Queue triggered abilities.
        self.queue_abilities(abl_queue, n_iter)

        # Print ability queue.
        if verbose and len(abl_queue.queue) > 0:
            print('Ability Queue:')
            for abl in abl_queue.queue:
                print(abl[1])
            print()
            
        # Reset all trigger dictionaries.
        self.reset_triggers()

        # Remove dead animals and shift live animals to front, but preserve dead animals' positions.
        did_clean = self.clean(verbose)
        if verbose and did_clean: print()
        
        # Use a queued ability.
        if not abl_queue.empty():
            _, abl, gamestate, in_team1, n_triggers = abl_queue.get_nowait()
            
            # tiger check
            
            own_team = gamestate.team1 if in_team1 else gamestate.team2
            behind_pos = abl.owner.pos + 1
            if isinstance(self, Battle) and behind_pos < len(own_team) and own_team[behind_pos].species == 'Tiger':
                tiger_lvl = own_team[behind_pos].lvl
            else: 
                tiger_lvl = None
                
            # temp for debugging
            # print(abl.owner, abl, abl.n_triggers)
            for _ in range(n_triggers):
                abl.use(gamestate, in_team1, verbose)
                if tiger_lvl is not None:
                    if verbose: print(f'Repeating {abl.effect} as lvl. {tiger_lvl}:')
                    tiger_abl = make_abl_list(abl.owner, *pets_df.loc[(abl.owner.species, tiger_lvl), 'Trigger':'N_charges'])
                    self.queue_abilities(abl_queue, n_iter+1)
                    self.clean(verbose)
                    tiger_abl[0].use(self, in_team1, verbose)
                    
            if verbose: print()
                
            self.process_triggers(verbose, abl_queue, n_iter+1)
        
        else:  
            # no more queued abilities
            return
        
    def queue_abilities(self, abl_queue, n_iter):
        did_queue = False
        pet_list = self.team1.pets if not isinstance(self, Battle) else self.team1.pets + self.team2.pets
        for pet in pet_list:
            in_team1 = pet in self.team1
            for i, abl in enumerate(pet.ability):
                n_triggers, triggerers = abl.trigger_func(self, in_team1, abl.target == 'Triggerer')                
                if n_triggers > 0:
                    did_queue = True
                    abl.triggerers = triggerers
                    # prioritize atk, then left-to-right, then own order, then random
                    priority = (n_iter, -pet.atk, -pet.pos-1 if in_team1 else pet.pos, i, np.random.sample())
                    abl_queue.put((priority, abl, self, in_team1, n_triggers))
        return did_queue
        
    def clean(self, verbose=False):
        """
        Remove dead pets from self.pets. Update position of dead pets to a 'half-position' (e.g. 1.5) in case
        Faint abilities still need to be activated.
        """
        did_clean = False
        for team in [self.team1, self.team2]:
            if isinstance(team, Team):
                for pet in team.pets[::-1]:
                    if pet.hp <= 0 or pet.atk <= 0:  # in case stats were swapped
                        if verbose: print(f'Cleaned {pet.species}.')
                        did_clean = True
                        team.dead_pets.append(team.drop(pet.pos))
        return did_clean
                
    def reset_triggers(self):
        def reset_helper(obj):
            for key in obj.trigger_dict.keys():
                obj.trigger_dict[key] = 0
        [reset_helper(pet) for pet in self.team1]
        if isinstance(self.team2, Team):
            [reset_helper(pet) for pet in self.team2]
        